<a href="https://colab.research.google.com/github/chenyujiehome/2021National-College-Students-Mathematical-Modeling-Contest/blob/main/case2_allstep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# prepare data



In [ ]:
## Imports
import numpy as np
import pandas as pd
import random
!pip install -q arize[AutoEmbeddings]

# Imports from Arize
from arize.pandas.logger import Client
from arize.utils.types import Environments, ModelTypes, EmbeddingColumnNames, Schema
from arize.pandas.embeddings.tabular_generators import EmbeddingGeneratorForTabularFeatures
random.seed(314159)  # Ensure repeatabiliy

from datetime import datetime, timedelta
## Sampling
replace = True             # Use replacement in sampling

# Anomaly Creation
anomaly_fraction    = 0.03  # Fraction of the anomalies (0.05 = 5% of the data are anomalies in window.)
anomaly_quantile    = 0.60  # Quantile about which the anomaly is centered. (Median=0.50)

anomaly_cols        = ['AveRooms', 'AveOccup' , 'Population', 'AveBedrms'] # X_cols used to create anomalies
## Arize Setings

SPACE_KEY = 'd1e0b68'
API_KEY   = '8e035808926c673e2fa'
# Time Window Defintiions

# Useful Definitions
now               = datetime.now()
today             = now.replace( hour=0, minute=0, second=0, microsecond=0 )
yesterday_end     = today - timedelta( microseconds=1 )

# this_month_start  = now.replace( day=1, hour=0, minute=0, second=0, microsecond=0 )
# last_month_end    = this_month_start - timedelta( microseconds = 1 )
# last_month_start  = last_month_end.replace( day=1, hour=0, minute=0, second=0, microsecond=0 )

# prod window
current_prod_day = datetime.today()
prod_start       = today - timedelta( days=20 ) # current_prod_day - timedelta( days=20 )
prod_end         = yesterday_end

print( "production is from: " + str( prod_start.date() ) + " through " + str( prod_end.date() ) )

# anomaly occurs from the 24 thru 26 of the month
anom_start     = ( prod_end - timedelta( days=6 ) ).replace( hour=0, minute=0, second=0)
anom_end       = ( prod_end - timedelta( days=3 ) ).replace(second=0, microsecond=0, minute=0, hour=0 ) - timedelta( microseconds=1 )

print( "Anomalies is from: " + str( anom_start.date() ) + " through " + str( anom_end.date() ) )
from sklearn.datasets import fetch_california_housing

data = fetch_california_housing(as_frame=True).frame

y_col  = 'MedHouseVal'               # Column Name of dependent variable `y`
X_cols = data.columns.drop(y_col)    # Column Name of independent variables of `X`

# data.describe()  # Describe Data
# data.hist(bins=30, figsize=(15, 10))  # Show Histograms
# Generate Data `train` and `prod` by sampling from `data` with replacement

## Training Data
train = data.sample( n=5000, replace=replace ).reset_index()
train['timestamp'] = current_prod_day  # This doesn't matter it could be anytime out o

## Production Data
prod_obs_list = []    # list of observation to be appended
day_to_set = prod_end

while day_to_set >= prod_start:
  #Randomize daily volume except for anomalies (for convenience)
  if ( day_to_set >= anom_start ) and (day_to_set <= anom_end ):
      daily_volume = 1000
  else:
    # 500-2000 productions obs / day
    daily_volume = random.randint(500, 2000)

  #Create a days worth of dataframe data as random sample from raw day
  data_day = data.sample(n=daily_volume, replace=replace)
  data_day['timestamp'] = day_to_set
  prod_obs_list.append(data_day)

  day_to_set = day_to_set - timedelta(days=1)  # Iterate backwards.

  prod = pd.concat( prod_obs_list ).reset_index()
## Create Anomalies
# See settings for: anomaly_quantile, anomaly_fraction, anomaly_cols

total_volume_anomaly_days = len( prod[( prod['timestamp'] >=  anom_start ) & ( prod['timestamp'] <= anom_end ) ] )
total_anomaly_days = (anom_end - anom_start).days + 1

n_anomalies_per_day = int(anomaly_fraction * (total_volume_anomaly_days/total_anomaly_days) * (1 / (1-anomaly_fraction)))
n_anomalies =   n_anomalies_per_day * total_anomaly_days


anomalies = pd.DataFrame()  # DataFrame to hold each anomaly

# Make a DF by repeating that observation
for i in range( n_anomalies ):
  row = prod.sample(n=1,replace=True)
  anomalies = pd.concat( [ anomalies, row ] )

anomalies = anomalies.reset_index( drop = True )


# Set `anomaly` flag
train['anomaly'] = False
prod['anomaly'] = False
anomalies['anomaly'] = True
# Generate Timestamp.
anomalies['timestamp'] = pd.to_datetime(
    pd.Series(
      np.random.uniform( anom_start.timestamp(), anom_end.timestamp(), n_anomalies )
    )
    , unit='s'
  )

# Generate Anomalies
for x in anomaly_cols:
  quantile_for_column = data[x].quantile( anomaly_quantile )
  print("Column " + x + " quantile " + str( round(quantile_for_column, 3)) +" setting to +/- 0.95/1.05 is " + str( round(0.95*quantile_for_column,3) ) + " " + str( round(1.05*quantile_for_column, 3)) )
  anomalies[x] = quantile_for_column* np.random.uniform( 0.95, 1.05, anomalies.shape[0])

print( "Anomalies created: " + str(anomalies.shape[0]) )
if sum(prod.anomaly) == 0:              # Ensures idempotency, this is done one time.
  prod=pd.concat( [prod, anomalies] )

prod = prod.sample(frac=1, axis=0).reset_index(drop=True)
## (Optional) Train Model
from sklearn.ensemble import RandomForestRegressor

# target = 'MedHouseVal'
drop_col = [ 'timestamp', 'index' ]
drop_col.append( y_col )

# x_cols = train.columns.drop( drop_col )

y = train[ y_col ]
X = train[ X_cols ]

# Train Model
regr = RandomForestRegressor(max_depth=5, random_state=0)
regr.fit(X, y)
pred_col = 'prediction'

train[ pred_col ] = regr.predict( train[ X_cols ] )
prod[ pred_col ] = regr.predict( prod[ X_cols ] )
# Install 'arize' package


arize_client = Client(space_key=SPACE_KEY, api_key=API_KEY)

# generate llama embedding

In [ ]:

import torch
from tqdm import tqdm
import pandas as pd
from transformers import AutoTokenizer, AutoModel

# 预先加载的模型和分词器
model_name = "meta-llama/Llama-2-7b-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name, token=api)
model = AutoModel.from_pretrained(model_name, token=api)
# 加载DataFrame
df = prod

# 确定列名
column_names = [f'vec_{i}' for i in range(4096)]  # 假设嵌入的长度是4096

# 确定是否有可用的GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 将模型加载到GPU
model = model.to(device)

# 确定从哪一行开始处理
start_row = 0
output_file = "embeddings.csv"

# 如果文件已存在，计算已处理的行数
try:
    with open(output_file, "r") as f:
        start_row = len(f.readlines()) - 1  # 减去标题行
    append_write = 'a'  # 追加模式
except FileNotFoundError:
    append_write = 'w'  # 写入模式

# 开始处理
with open(output_file, append_write) as f:
    if append_write == 'w':
        # 写入列名
        f.write(",".join(column_names) + "\n")

    # 使用tqdm显示进度条
    for index, row in tqdm(df.iterrows(), total=df.shape[0], initial=start_row):
        if index < start_row:
            continue  # 跳过已处理的行

        prompt = "".join([f"the {col} is {row[col]}." for col in df.columns])
        tokens = tokenizer(prompt, return_tensors='pt')
        tokens = {k: v.to(device) for k, v in tokens.items()}

        with torch.no_grad():
            outputs = model(**tokens)

        embeddings = torch.mean(outputs.last_hidden_state, 1)
        embeddings_np = embeddings.cpu().numpy()[0]
        embeddings_str = ",".join(map(str, embeddings_np))
        f.write(embeddings_str + "\n")

        torch.cuda.empty_cache()  # 清理GPU缓存


In [ ]:
import pandas as pd
df=pd.read_csv("embeddings.csv")
# 假设 'df' 是您原始的 DataFrame
list_of_vectors = df.values.tolist()
prod['tabular_vector'] = list_of_vectors
df.to_pickle("prod.pkl")

# send to arize

In [ ]:
obs_in_anomaly_window = len(            # Observations in anomaly window
    prod[
         ( prod['timestamp'] >= anom_start) & ( prod['timestamp'] <= anom_end )
        ]
)

actual_anomalies = sum(prod.anomaly)    # Number of anomalies

final_anomaly_fraction = actual_anomalies/obs_in_anomaly_window

print( "days with anomalies: " + str( obs_in_anomaly_window ) )
print( "actual anomalies: " + str( actual_anomalies) )
print( "fraction anolalies: " + str( np.round(final_anomaly_fraction, 4) ) )

# Lets create a model ID based on time for this run - as a string
#import datetime

model_id = "anomaly-" + \
  datetime.now().strftime( "%Y_%m_%d_%H:%m" ) + \
  "-frac_anom_" + "{:.2f}".format(round(final_anomaly_fraction, 2)) + \
  "-quant_" + str(anomaly_quantile) + \
  "-anom_columns_" + str(len(anomaly_cols))

print( "model name is: \n  " + model_id )
# Model ID

model_version = "1.3"
model_type = ModelTypes.REGRESSION

if SPACE_KEY == "SPACE_KEY" or API_KEY == "API_KEY":
    raise ValueError("❌ NEED TO CHANGE SPACE AND/OR API_KEY")
else:
    print("✅ Import and Setup Arize Client Done! Now we can start using Arize!")
    embedding_features = {
    # Dictionary keys will be name of embedding feature in the app
    "tabular embedding": EmbeddingColumnNames(
        vector_column_name = "tabular_vector",
        #  data_column_name="prompt"
    ),
}

# Define a Schema() object for Arize to pick up data from the correct columns for logging
schema = Schema(
    timestamp_column_name           = 'timestamp',
    prediction_id_column_name       = 'index',
    prediction_label_column_name    = pred_col ,
    actual_label_column_name        = y_col,
    feature_column_names            = list(X_cols) + ['index'],
    embedding_feature_column_names  = embedding_features,
    tag_column_names                = [ 'anomaly' ]
)
# Logging Production DataFrame
prod = prod.reset_index( drop=True)
response = arize_client.log(
    dataframe     = prod,
    model_id      = model_id,
    model_version = model_version,
    model_type    = model_type,
    environment   = Environments.PRODUCTION,
    schema        = schema,
)

print( "prod data with " + str(prod.shape[0]) + " rows sent at: " + str( datetime.now() ) )
print( "Anomalies are found between: " + str(anom_start.date()) + " thru " + str(anom_end.date()) )